In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import  nltk.translate.bleu_score as bleu

In [2]:
path_to_zip = tf.keras.utils.get_file(
    'cornell_movie_dialogs.zip',
    origin=
    'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_dataset = os.path.join(
    os.path.dirname(path_to_zip), "cornell movie-dialogs corpus")

path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
path_to_movie_conversations = os.path.join(path_to_dataset,
                                           'movie_conversations.txt')

In [3]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  sentence = '<start> ' + sentence + ' <end>'
  return sentence


def load_conversations(num1,num2):
  # dictionary of line id to text
  id2line = {}
  with open(path_to_movie_lines, errors='ignore') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, 'r') as file:
    lines = file.readlines()
  for line in lines[num1:num2]:
    parts = line.replace('\n', '').split(' +++$+++ ')
    # get conversation in a list of line ID
    conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]
    for i in range(len(conversation) - 1):
      inputs.append(preprocess_sentence(id2line[conversation[i]]))
      outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))
#       if len(inputs) >= MAX_SAMPLES:
#         return inputs, outputs
  return inputs, outputs

In [4]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> puedo tomar prestado este libro ? <end>'


In [4]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [5]:
def load_dataset(num1,num2):
  # creating cleaned input, output pairs
  inp_lang, targ_lang = load_conversations(num1,num2)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [6]:
num1 = 0
num2 = 5000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(num1,num2)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [10]:
inp_lang.index_word

{1: '.',
 2: '<start>',
 3: '<end>',
 4: ',',
 5: 'you',
 6: 'i',
 7: '?',
 8: 'the',
 9: 'to',
 10: 'a',
 11: 's',
 12: 'it',
 13: 't',
 14: 'that',
 15: 'and',
 16: 'what',
 17: 'of',
 18: '!',
 19: 'me',
 20: 'in',
 21: 'is',
 22: 'we',
 23: 'he',
 24: 'this',
 25: 'for',
 26: 'don',
 27: 'do',
 28: 'm',
 29: 'my',
 30: 'your',
 31: 'have',
 32: 'know',
 33: 'not',
 34: 'was',
 35: 'be',
 36: 'are',
 37: 're',
 38: 'on',
 39: 'no',
 40: 'with',
 41: 'but',
 42: 'can',
 43: 'they',
 44: 'like',
 45: 'just',
 46: 'all',
 47: 'about',
 48: 'get',
 49: 'how',
 50: 'so',
 51: 'if',
 52: 'there',
 53: 'here',
 54: 'll',
 55: 'out',
 56: 'up',
 57: 'him',
 58: 'think',
 59: 'got',
 60: 'want',
 61: 'one',
 62: 'go',
 63: 've',
 64: 'why',
 65: 'at',
 66: 'now',
 67: 'she',
 68: 'yes',
 69: 'did',
 70: 'who',
 71: 'right',
 72: 'well',
 73: 'her',
 74: 'see',
 75: 'going',
 76: 'd',
 77: 'tell',
 78: 'as',
 79: 'been',
 80: 'some',
 81: 'where',
 82: 'good',
 83: 'oh',
 84: 'when',
 85: 'ma

In [8]:
max_length_targ

242

In [9]:
max_length_inp

273

In [10]:
input_tensor_train = input_tensor
target_tensor_train = target_tensor

In [11]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [12]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
42 ----> can
22 ----> we
121 ----> make
24 ----> this
1309 ----> quick
7 ----> ?
4914 ----> roxanne
4915 ----> korrine
15 ----> and
4916 ----> andrew
4917 ----> barrett
36 ----> are
481 ----> having
87 ----> an
1617 ----> incredibly
4918 ----> horrendous
989 ----> public
536 ----> break
56 ----> up
38 ----> on
8 ----> the
4919 ----> quad
1 ----> .
235 ----> again
1 ----> .
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
68 ----> well
4 ----> ,
5 ----> i
159 ----> thought
24 ----> we
81 ----> d
321 ----> start
41 ----> with
4838 ----> pronunciation
4 ----> ,
51 ----> if
14 ----> that
11 ----> s
102 ----> okay
41 ----> with
6 ----> you
1 ----> .
3 ----> <end>


In [13]:
BUFFER_SIZE = 20000 #len(input_tensor_train)
BATCH_SIZE = 4 #64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 4 #512
units = 64 # 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [14]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([4, 273]), TensorShape([4, 242]))

In [15]:
# Write the encoder and decoder model

In [16]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [17]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (4, 273, 64)
Encoder Hidden state shape: (batch size, units) (4, 64)


In [18]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [19]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (4, 64)
Attention weights shape: (batch_size, sequence_length, 1) (4, 273, 1)


In [20]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [21]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (4, 9430)


In [22]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [23]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [24]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [25]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.6523
Epoch 1 Batch 100 Loss 0.2041
Epoch 1 Batch 200 Loss 0.5143
Epoch 1 Batch 300 Loss 0.3473
Epoch 1 Batch 400 Loss 0.1570
Epoch 1 Batch 500 Loss 0.2720
Epoch 1 Batch 600 Loss 0.2232
Epoch 1 Batch 700 Loss 0.2171
Epoch 1 Batch 800 Loss 0.2330
Epoch 1 Batch 900 Loss 0.2187
Epoch 1 Batch 1000 Loss 0.2210
Epoch 1 Batch 1100 Loss 0.3514
Epoch 1 Batch 1200 Loss 0.6012
Epoch 1 Batch 1300 Loss 0.2414
Epoch 1 Batch 1400 Loss 0.1896
Epoch 1 Batch 1500 Loss 0.2836
Epoch 1 Batch 1600 Loss 0.1859
Epoch 1 Batch 1700 Loss 0.2657
Epoch 1 Batch 1800 Loss 0.5869
Epoch 1 Batch 1900 Loss 0.2211
Epoch 1 Batch 2000 Loss 0.4091
Epoch 1 Batch 2100 Loss 0.2966
Epoch 1 Batch 2200 Loss 0.1534
Epoch 1 Batch 2300 Loss 0.6176
Epoch 1 Batch 2400 Loss 0.2589
Epoch 1 Batch 2500 Loss 0.7386
Epoch 1 Batch 2600 Loss 0.1870
Epoch 1 Batch 2700 Loss 0.2606
Epoch 1 Batch 2800 Loss 0.1379
Epoch 1 Batch 2900 Loss 0.2756
Epoch 1 Batch 3000 Loss 0.1089
Epoch 1 Batch 3100 Loss 0.3971
Epoch 1 Batch 3200 L

In [26]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [27]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [53]:
def chatbot(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))
  return result

In [54]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [55]:
chatbot('who are you?')

Input: <start> who are you ? <end>
Predicted translation: i m a little . <end> 


'i m a little . <end> '

In [56]:
chatbot('what did you eat for breakfast?')

Input: <start> what did you eat for breakfast ? <end>
Predicted translation: i m a little . <end> 


'i m a little . <end> '

In [57]:
chatbot('did you finish your homework?')

Input: <start> did you finish your homework ? <end>
Predicted translation: i m a little . <end> 


'i m a little . <end> '

In [58]:
questions_val, answers_val = load_conversations(80001,80500)
len(questions_val),len(answers_val)

(1634, 1634)

In [68]:
# bleuplus = 0

# for ques_, ans_ in zip(questions_val[0,1],answers_val[0,1]):
#     predicted = chatbot(ques_)
#     print('Real Response: {}'.format(ans_))
#     bleu_ = bleu.sentence_bleu([ans_.split()], predicted.split())
#     print("BLEU Score: ", bleu_)
#     print("----" *20)

#     bleuplus += bleu_

In [82]:
bleuplus = 0

for i in [0,1,4,5,11,13,1000,1002,500,504,700,749,22,90,91]:
    ques_ = questions_val[i]
    ans_ = answers_val[i]
    predicted = chatbot(ques_)
    print('Real Response: {}'.format(ans_))
    bleu_ = bleu.sentence_bleu([ans_.split()], predicted.split())
    print("BLEU Score: ", bleu_)
    print("----" *20)

    bleuplus += bleu_

Input: <start> start what is it , son ? end <end>
Predicted translation: i m a little . <end> 
Real Response: <start> two more hogs got the fever . <end>
BLEU Score:  4.597292750167502e-155
--------------------------------------------------------------------------------
Input: <start> start who s he ? end <end>
Predicted translation: i m a little . <end> 
Real Response: <start> best we move that pig . <end>
BLEU Score:  5.431059661402121e-155
--------------------------------------------------------------------------------
Input: <start> start three . end <end>
Predicted translation: i m a little . <end> 
Real Response: <start> three ? that ain t bad considerin . <end>
BLEU Score:  3.891524296988902e-155
--------------------------------------------------------------------------------
Input: <start> start that fella come by . . . tom . end <end>
Predicted translation: i m a little . <end> 
Real Response: <start> tom ? <end>
BLEU Score:  1.1640469867513693e-231
---------------------------

In [83]:
bleuplus/15

2.1315202437142175e-155

In [84]:
chatbot('how are you doing?')

Input: <start> how are you doing ? <end>
Predicted translation: i m a little . <end> 


'i m a little . <end> '